# Sequence2Sequence Model

## Preparing the Environment


In [ ]:
#Google Colab - Drive Mounting
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
#Install missing library keras-nlp
!pip install -q keras-nlp

     |████████████████████████████████| 151 kB 14.0 MB/s 
     |████████████████████████████████| 5.8 MB 70.4 MB/s 
     |████████████████████████████████| 588.3 MB 7.1 kB/s 
     |████████████████████████████████| 5.9 MB 56.2 MB/s 
     |████████████████████████████████| 578.1 MB 30 kB/s 
     |████████████████████████████████| 578.0 MB 20 kB/s 
     |████████████████████████████████| 4.6 MB 52.7 MB/s 


In [ ]:
# Import the libraries
import os
import keras_nlp
import tensorflow as tf
from tensorflow import keras
import numpy as np

## Data Preprocessing and Parameters Initialization

Reference: https://keras.io/guides/keras_nlp/transformer_pretraining/

In [ ]:
# Download of the vocabulary from BERT: Bert-uncased
vocab_file = keras.utils.get_file(
    origin="https://storage.googleapis.com/tensorflow/keras-nlp/examples/bert/bert_vocab_uncased.txt",
)
#Initialization of the Word Tokenizer, with a given vocabulary and a sequence length
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab_file, sequence_length=SEQ_LENGTH,
)

231508/231508 [==============================] - 0s 1us/step


In [ ]:
# Preprocessing paramseters
PRETRAINING_BATCH_SIZE = 128
FINETUNING_BATCH_SIZE = 32
SEQ_LENGTH = 128
MASK_RATE = 0.25
PREDICTIONS_PER_SEQ = 32

# Model paramerters
NUM_LAYERS = 2
MODEL_DIM = 256
INTERMEDIATE_DIM = 512
NUM_HEADS = 4
DROPOUT = 0.1
NORM_EPSILON = 1e-5

# Training paramseters
PRETRAINING_LEARNING_RATE = 5e-4
PRETRAINING_EPOCHS = 15 
FINETUNING_LEARNING_RATE = 5e-5
FINETUNING_EPOCHS = 3

In [ ]:
# Load the data
gutenberg_train_ds = (
    tf.data.TextLineDataset('path_to_pretraining_train_data/train-all.csv')
    .filter(lambda x: tf.strings.length(x) > 100)
    .batch(PRETRAINING_BATCH_SIZE)
)
gutenberg_val_ds = (
    tf.data.TextLineDataset('path_to_pretraining_test_data/test-all.csv')
    .filter(lambda x: tf.strings.length(x) > 100)
    .batch(PRETRAINING_BATCH_SIZE)
)

In [ ]:
def preprocess(inputs):
    """
    Given a text input, 
    return a set of features containing the masked words, 
    the true labels of these masked word and their weights
    """
    inputs = tokenizer(inputs)
    outputs = masker(inputs)
    features = {
        "tokens": outputs["tokens"],
        "mask_positions": outputs["mask_positions"],
    }
    labels = outputs["mask_ids"]
    weights = outputs["mask_weights"]
    return features, labels, weights

# Create the Masker object
masker = keras_nlp.layers.MLMMaskGenerator(
    vocabulary_size=tokenizer.vocabulary_size(),
    mask_selection_rate=MASK_RATE,
    mask_selection_length=PREDICTIONS_PER_SEQ,
    mask_token_id=tokenizer.token_to_id("[MASK]"),
)


# Preprocess the data
pretrain_ds = gutenberg_train_ds.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)
pretrain_val_ds = gutenberg_val_ds.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

# Display a single element of the dataset
print(pretrain_ds.take(1).get_single_element())

## Build Model

In [ ]:
# Define input type
inputs = keras.Input(shape=(SEQ_LENGTH,), dtype=tf.int32)

# Embdding layer
embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=tokenizer.vocabulary_size(),
    sequence_length=SEQ_LENGTH,
    embedding_dim=MODEL_DIM,
)
outputs = embedding_layer(inputs)

# Normalization and Dropout
outputs = keras.layers.LayerNormalization(epsilon=NORM_EPSILON)(outputs)
outputs = keras.layers.Dropout(rate=DROPOUT)(outputs)

# Create encoder blocks
for i in range(NUM_LAYERS):
    outputs = keras_nlp.layers.TransformerEncoder(
        intermediate_dim=INTERMEDIATE_DIM,
        num_heads=NUM_HEADS,
        dropout=DROPOUT,
        layer_norm_epsilon=NORM_EPSILON,
    )(outputs)

# Create decoder blocks
for i in range(NUM_LAYERS):
    outputs = keras_nlp.layers.TransformerDecoder(
        intermediate_dim=INTERMEDIATE_DIM,
        num_heads=NUM_HEADS,
        dropout=DROPOUT,
        layer_norm_epsilon=NORM_EPSILON,
    )(outputs)


#Build model and display
encoder_model = keras.Model(inputs, outputs)
encoder_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128)]             0         
                                                                 
 token_and_position_embeddin  (None, 128, 256)         7846400   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 layer_normalization (LayerN  (None, 128, 256)         512       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 128, 256)          0         
                                                                 
 transformer_encoder (Transf  (None, 128, 256)         527104    
 ormerEncoder)                                               

## Model Training

In [ ]:
# Adding masked language model head
inputs = {
    "tokens": keras.Input(shape=(SEQ_LENGTH,), dtype=tf.int32),
    "mask_positions": keras.Input(shape=(PREDICTIONS_PER_SEQ,), dtype=tf.int32),
}

#Create tensorboard callback
logdir = "path_to_save_execution_information" #+ datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Encode the input tokens
encoded_tokens = encoder_model(inputs["tokens"])

# Predict output for each masked word
outputs = keras_nlp.layers.MLMHead(
    embedding_weights=embedding_layer.token_embedding.embeddings, activation="softmax",
)(encoded_tokens, mask_positions=inputs["mask_positions"])

# Compile our model
pretraining_model = keras.Model(inputs, outputs)
pretraining_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=PRETRAINING_LEARNING_RATE),
    weighted_metrics=["sparse_categorical_accuracy"],
    jit_compile=True,
)

In [ ]:
# Model Training
pretraining_model.fit(
    pretrain_ds,  validation_data=pretrain_val_ds, epochs=PRETRAINING_EPOCHS,callbacks=[tensorboard_callback],
)

Epoch 1/15
2074/2074 [==============================] - 489s 227ms/step - loss: 2.8612 - sparse_categorical_accuracy: 0.1726 - val_loss: 2.5227 - val_sparse_categorical_accuracy: 0.2676
Epoch 2/15
2074/2074 [==============================] - 471s 227ms/step - loss: 2.2215 - sparse_categorical_accuracy: 0.2983 - val_loss: 2.2563 - val_sparse_categorical_accuracy: 0.3194
Epoch 3/15
2074/2074 [==============================] - 468s 226ms/step - loss: 2.0436 - sparse_categorical_accuracy: 0.3348 - val_loss: 2.1412 - val_sparse_categorical_accuracy: 0.3442
Epoch 4/15
2074/2074 [==============================] - 493s 238ms/step - loss: 1.9477 - sparse_categorical_accuracy: 0.3550 - val_loss: 2.0689 - val_sparse_categorical_accuracy: 0.3583
Epoch 5/15
2074/2074 [==============================] - 468s 226ms/step - loss: 1.8831 - sparse_categorical_accuracy: 0.3691 - val_loss: 2.0207 - val_sparse_categorical_accuracy: 0.3701
Epoch 6/15
2074/2074 [==============================] - 495s 239ms/ste

In [ ]:
# Save this base model for further finetuning.
encoder_model.save("seq2seq_all")

In [ ]:
#Load Tensorboard
%reload_ext tensorboard
%tensorboard --logdir="path_to_save_execution_information"

In [ ]:
#Code to automatically stop the run time for Google Colab
import time
time.sleep(60)
from google.colab import runtime
runtime.unassign()